In [1]:
# importo las librerias a utilizar
import pandas as pd
import numpy as np
import geopandas as gpd
from scipy import spatial
#import matplotlib.pyplot as plt

#import shapely # genera las figuras geometricas
#import descartes # relaciona shapely con matplotlib
#import pyproj # proyecciones. Transformar coordenadas

In [2]:
# Levanto los datos de properratti
data_location = "./data/properatti.csv"
dataP = pd.read_csv(data_location, sep=",")
dataP.head(2)


,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,...,40.0,1127.272727,1550.0,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...


In [4]:
# Tipos de columnas
dataP.dtypes

Unnamed: 0                      int64
operation                      object
property_type                  object
place_name                     object
place_with_parent_names        object
country_name                   object
state_name                     object
geonames_id                   float64
lat-lon                        object
lat                           float64
lon                           float64
price                         float64
currency                       object
price_aprox_local_currency    float64
price_aprox_usd               float64
surface_total_in_m2           float64
surface_covered_in_m2         float64
price_usd_per_m2              float64
price_per_m2                  float64
floor                         float64
rooms                         float64
expenses                      float64
properati_url                  object
description                    object
title                          object
image_thumbnail                object
dtype: objec

In [5]:
# Porcentaje de nulos
dataP.isnull().sum() / dataP.shape[0]

Unnamed: 0                    0.000000
operation                     0.000000
property_type                 0.000000
place_name                    0.000190
place_with_parent_names       0.000000
country_name                  0.000000
state_name                    0.000000
geonames_id                   0.154405
lat-lon                       0.425260
lat                           0.425260
lon                           0.425260
price                         0.168372
currency                      0.168380
price_aprox_local_currency    0.168372
price_aprox_usd               0.168372
surface_total_in_m2           0.324435
surface_covered_in_m2         0.164222
price_usd_per_m2              0.433947
price_per_m2                  0.276869
floor                         0.934837
rooms                         0.609058
expenses                      0.882346
properati_url                 0.000000
description                   0.000016
title                         0.000000
image_thumbnail          

In [22]:
# Reviso las diferentes provincias con datos
dataP.state_name.value_counts()

Capital Federal                 32316
Bs.As. G.B.A. Zona Norte        25560
Bs.As. G.B.A. Zona Sur          13952
Córdoba                         12069
Santa Fe                        10172
Buenos Aires Costa Atlántica    10006
Bs.As. G.B.A. Zona Oeste         9322
Buenos Aires Interior            2291
Río Negro                         808
Neuquén                           733
Mendoza                           681
Tucumán                           674
Corrientes                        583
Misiones                          464
Entre Ríos                        369
Salta                             278
Chubut                            259
San Luis                          252
La Pampa                          157
Formosa                            65
Chaco                              57
San Juan                           40
Tierra Del Fuego                   31
Catamarca                          27
Jujuy                              26
Santa Cruz                         20
La Rioja    

In [3]:
# Obtengo los datos solo para CABA
maskCaba = dataP.state_name == 'Capital Federal'
dataP_caba = dataP.loc[maskCaba]
# Porcentaje de nulos en CABA
dataP_caba.isnull().sum() / dataP_caba.shape[0]

Unnamed: 0                    0.000000
operation                     0.000000
property_type                 0.000000
place_name                    0.000000
place_with_parent_names       0.000000
country_name                  0.000000
state_name                    0.000000
geonames_id                   0.039795
lat-lon                       0.264451
lat                           0.264451
lon                           0.264451
price                         0.102055
currency                      0.102055
price_aprox_local_currency    0.102055
price_aprox_usd               0.102055
surface_total_in_m2           0.181922
surface_covered_in_m2         0.072379
price_usd_per_m2              0.274848
price_per_m2                  0.154475
floor                         0.906888
rooms                         0.525777
expenses                      0.796014
properati_url                 0.000000
description                   0.000000
title                         0.000000
image_thumbnail          

In [4]:
# Reviso los casos con precio y superficie null
dataP_cabaR = dataP_caba.loc[:,['price','currency','price_aprox_local_currency','price_aprox_usd',\
                                   'surface_total_in_m2', 'surface_covered_in_m2', 'price_usd_per_m2', 'price_per_m2']]

mask_dataP_cabaNull = dataP_cabaR.isnull()
serieNull = mask_dataP_cabaNull.all(axis=1)
#print(mask_dataP_cabaNull.all(axis=1))
#print(serieNull[serieNull])

# Tengo 22 filas donde todos los datos son nulos - genero un nuevo dataframe "dataP_cabaRsN" sin esas filas

print(dataP_cabaR.shape)
dataP_cabaRsN = dataP_cabaR.dropna(how='all')
print(dataP_cabaRsN.shape)

dataP_cabaSn = dataP_caba.dropna(subset=['price','currency','price_aprox_local_currency','price_aprox_usd',\
                                   'surface_total_in_m2', 'surface_covered_in_m2', 'price_usd_per_m2', 'price_per_m2'])
print(dataP_cabaSn.shape)

(32316, 8)
(32094, 8)
(22317, 26)


In [8]:
dataP_cabaRsN.head(3)

,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2
0,62000.0,USD,1093959.0,62000.0,55.0,40.0,1127.272727,1550.000000
2,72000.0,USD,1270404.0,72000.0,55.0,55.0,1309.090909,1309.090909
3,95000.0,USD,1676227.5,95000.0,NaN,NaN,NaN,NaN


In [5]:
# Sumo el dataFrame de estaciones de subte
data_location = "./data/estaciones-de-subte.csv"
dataSubte = pd.read_csv(data_location, sep=',')
dataSubte.head(3)

,long,lat,id,estacion,linea
0,-58.398928,-34.635750,1.0,CASEROS,H
1,-58.400970,-34.629376,2.0,INCLAN - MEZQUITA AL AHMAD,H
2,-58.402323,-34.623092,3.0,HUMBERTO 1°,H


In [6]:
#Transformo el dataFrame en geoDataframe
geo_subte = gpd.GeoDataFrame(dataSubte,geometry=gpd.points_from_xy(dataSubte.long, dataSubte.lat))
geo_subte.head(3)

,long,lat,id,estacion,linea,geometry
0,-58.398928,-34.635750,1.0,CASEROS,H,POINT (-58.39893 -34.63575)
1,-58.400970,-34.629376,2.0,INCLAN - MEZQUITA AL AHMAD,H,POINT (-58.40097 -34.62938)
2,-58.402323,-34.623092,3.0,HUMBERTO 1°,H,POINT (-58.40232 -34.62309)


In [7]:
#Transformo el dataFrame de propiedades caba en geoDataframe
geometria = gpd.points_from_xy(dataP_cabaSn.lon, dataP_cabaSn.lat)
geo_dataP_cabaSn = gpd.GeoDataFrame(dataP_cabaSn ,geometry=geometria)
geo_dataP_cabaSn.head(4)

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail,geometry
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,...,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...,POINT (-58.50884 -34.66182)
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,...,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...,POINT (-58.52298 -34.65226)
7,7,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,"-34.5598729,-58.443362",-34.559873,...,3066.666667,3450.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bot_venta_depart...,EXCELENTE MONOAMBIENTE A ESTRENAR AMPLIO SUPER...,JOSE HERNANDEZ 1400 MONOAMBIENTE ESTRENAR CAT...,https://thumbs4.properati.com/1/IHxARynlr8sPEW...,POINT (-58.44336 -34.55987)
8,8,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,"-34.5598729,-58.443362",-34.559873,...,3000.000000,3250.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bou_venta_depart...,EXCELENTE DOS AMBIENTES ESTRENAR AMPLIO SUPER...,"JOSE HERNANDEZ 1400 DOS AMBIENTES ESTRENAR ,...",https://thumbs4.properati.com/2/J3zOjgaFHrkvnv...,POINT (-58.44336 -34.55987)


In [8]:
#dejo solo las columnas que me dan valor.
geo_dataP_cabaSn = geo_dataP_cabaSn.loc[:,['operation', 'property_type', 'place_name', 'lat', 'lon', 'geometry', 'price', \
                                        'currency', 'price_aprox_local_currency', 'price_aprox_usd', 'surface_total_in_m2', \
                                        'surface_covered_in_m2', 'price_usd_per_m2', 'price_per_m2', 'floor', 'rooms', \
                                         'expenses', 'description']]


geo_dataP_cabaSn.head(3)


,operation,property_type,place_name,lat,lon,geometry,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,description
0,sell,PH,Mataderos,-34.661824,-58.508839,POINT (-58.50884 -34.66182),62000.0,USD,1093959.0,62000.0,55.0,40.0,1127.272727,1550.000000,NaN,NaN,NaN,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,..."
2,sell,apartment,Mataderos,-34.652262,-58.522982,POINT (-58.52298 -34.65226),72000.0,USD,1270404.0,72000.0,55.0,55.0,1309.090909,1309.090909,NaN,NaN,NaN,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...
7,sell,apartment,Belgrano,-34.559873,-58.443362,POINT (-58.44336 -34.55987),138000.0,USD,2434941.0,138000.0,45.0,40.0,3066.666667,3450.000000,NaN,NaN,NaN,EXCELENTE MONOAMBIENTE A ESTRENAR AMPLIO SUPER...


In [9]:
def obtenerCaracteristicas(valor):
    valor = valor.lower()
    caracteristica = '|'
    if valor.find('piscina')>-1 or valor.find('pileta'):
        caracteristica = caracteristica  + 'Piscina|'
    
    if valor.find('estrenar')>-1:
        caracteristica = caracteristica + 'Estrenar|'
    
    return caracteristica
    
    
geo_dataP_cabaSn['caracteristicas'] = geo_dataP_cabaSn.description.apply(lambda x: x if x is np.NaN else obtenerCaracteristicas(x))

geo_dataP_cabaSn.head(4)

,operation,property_type,place_name,lat,lon,geometry,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,description,caracteristicas
0,sell,PH,Mataderos,-34.661824,-58.508839,POINT (-58.50884 -34.66182),62000.0,USD,1093959.0,62000.0,55.0,40.0,1127.272727,1550.000000,NaN,NaN,NaN,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",|Piscina|
2,sell,apartment,Mataderos,-34.652262,-58.522982,POINT (-58.52298 -34.65226),72000.0,USD,1270404.0,72000.0,55.0,55.0,1309.090909,1309.090909,NaN,NaN,NaN,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,|Piscina|
7,sell,apartment,Belgrano,-34.559873,-58.443362,POINT (-58.44336 -34.55987),138000.0,USD,2434941.0,138000.0,45.0,40.0,3066.666667,3450.000000,NaN,NaN,NaN,EXCELENTE MONOAMBIENTE A ESTRENAR AMPLIO SUPER...,|Piscina|Estrenar|
8,sell,apartment,Belgrano,-34.559873,-58.443362,POINT (-58.44336 -34.55987),195000.0,USD,3440677.5,195000.0,65.0,60.0,3000.000000,3250.000000,NaN,NaN,NaN,EXCELENTE DOS AMBIENTES ESTRENAR AMPLIO SUPER...,|Piscina|Estrenar|


In [41]:
#Calculo la distancia menor a una estación de subte
geo_dataP_cabaSn["distancia_min"] = np.min(spatial.distance.cdist(
    geo_dataP_cabaSn[["lon", "lat"]], geo_subte[["long", "lat"]]
    ), axis=1).round(4)
#geo_dataP_cabaSn.sort_values(by='distancia_min', ascending=False).head(30)

In [70]:
# Separo las distancias en 5 categorias
distaQCat = pd.cut(geo_dataP_cabaSn.distancia_min,  [-1, 0.01, 0.05, 1], labels=['Cerca', 'Medio', 'Lejos'])
print(distaQCat)
geo_dataP_cabaSn['distanciaSubte'] = distaQCat
geo_dataP_cabaSn.head(3)


0         Medio
2         Lejos
7         Medio
8         Medio
13          NaN
          ...  
121157      NaN
121158    Cerca
121215      NaN
121217    Cerca
121219      NaN
Name: distancia_min, Length: 22317, dtype: category
Categories (3, object): ['Cerca' < 'Medio' < 'Lejos']


,operation,property_type,place_name,lat,lon,geometry,price,currency,price_aprox_local_currency,price_aprox_usd,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,description,caracteristicas,distancia_min,distanciaSubte
0,sell,PH,Mataderos,-34.661824,-58.508839,POINT (-58.50884 -34.66182),62000.0,USD,1093959.0,62000.0,...,40.0,1127.272727,1550.000000,NaN,NaN,NaN,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",|Piscina|,0.0500,Medio
2,sell,apartment,Mataderos,-34.652262,-58.522982,POINT (-58.52298 -34.65226),72000.0,USD,1270404.0,72000.0,...,55.0,1309.090909,1309.090909,NaN,NaN,NaN,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,|Piscina|,0.0575,Lejos
7,sell,apartment,Belgrano,-34.559873,-58.443362,POINT (-58.44336 -34.55987),138000.0,USD,2434941.0,138000.0,...,40.0,3066.666667,3450.000000,NaN,NaN,NaN,EXCELENTE MONOAMBIENTE A ESTRENAR AMPLIO SUPER...,|Piscina|Estrenar|,0.0102,Medio


In [71]:
#Agrupo el dataframe
dataP_cabaG = geo_dataP_cabaSn.groupby(by=['operation','property_type','place_name', 'distanciaSubte'])
dataP_cabaG.head(3)

,operation,property_type,place_name,lat,lon,geometry,price,currency,price_aprox_local_currency,price_aprox_usd,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,description,caracteristicas,distancia_min,distanciaSubte
0,sell,PH,Mataderos,-34.661824,-58.508839,POINT (-58.50884 -34.66182),62000.0,USD,1093959.00,62000.0,...,40.0,1127.272727,1550.000000,NaN,NaN,NaN,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",|Piscina|,0.0500,Medio
2,sell,apartment,Mataderos,-34.652262,-58.522982,POINT (-58.52298 -34.65226),72000.0,USD,1270404.00,72000.0,...,55.0,1309.090909,1309.090909,NaN,NaN,NaN,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,|Piscina|,0.0575,Lejos
7,sell,apartment,Belgrano,-34.559873,-58.443362,POINT (-58.44336 -34.55987),138000.0,USD,2434941.00,138000.0,...,40.0,3066.666667,3450.000000,NaN,NaN,NaN,EXCELENTE MONOAMBIENTE A ESTRENAR AMPLIO SUPER...,|Piscina|Estrenar|,0.0102,Medio
8,sell,apartment,Belgrano,-34.559873,-58.443362,POINT (-58.44336 -34.55987),195000.0,USD,3440677.50,195000.0,...,60.0,3000.000000,3250.000000,NaN,NaN,NaN,EXCELENTE DOS AMBIENTES ESTRENAR AMPLIO SUPER...,|Piscina|Estrenar|,0.0102,Medio
13,sell,apartment,Palermo Soho,NaN,NaN,POINT (nan nan),111700.0,USD,1970890.65,111700.0,...,30.0,2234.000000,3723.333333,NaN,1.0,NaN,Torre I Mondrian. 3 ambientes con terraza y d...,|Piscina|,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119319,sell,PH,San Nicolás,-34.604555,-58.380247,POINT (-58.38025 -34.60455),120000.0,USD,2117340.00,120000.0,...,47.0,2553.191489,2553.191489,NaN,2.0,NaN,PH tipo loft reciclado a nuevo en el barrio de...,|Piscina|,0.0005,Cerca
119365,sell,PH,Pompeya,-34.645127,-58.420705,POINT (-58.42071 -34.64513),180000.0,USD,3176010.00,180000.0,...,90.0,1022.727273,2000.000000,NaN,5.0,NaN,Excelente PH. 4 ambientes con aire acondiciona...,|Piscina|,0.0092,Cerca
119376,sell,PH,Agronomía,-34.603179,-58.475603,POINT (-58.47560 -34.60318),130000.0,USD,2293785.00,130000.0,...,65.0,2000.000000,2000.000000,NaN,3.0,NaN,"Adolfo P. Carranza 2200, planta baja, PH de 3 ...",|Piscina|,0.0213,Medio
119422,sell,PH,Pompeya,-34.645127,-58.420705,POINT (-58.42071 -34.64513),180000.0,USD,3176010.00,180000.0,...,90.0,1022.727273,2000.000000,NaN,5.0,NaN,Excelente PH. 5 ambientes con gran patio muy l...,|Piscina|,0.0092,Cerca


In [72]:
dataP_cabaG.price.mean()

operation  property_type  place_name        distanciaSubte
sell       PH             Abasto            Cerca             166010.0
                                            Medio                  NaN
                                            Lejos                  NaN
                          Agronomía         Cerca                  NaN
                                            Medio             180000.0
                                                                ...   
           store          Villa Urquiza     Medio             200000.0
                                            Lejos                  NaN
                          Villa del Parque  Cerca                  NaN
                                            Medio             386500.0
                                            Lejos             320000.0
Name: price, Length: 744, dtype: float64

In [74]:
from sentiment_analysis_spanish import sentiment_analysis

In [77]:
#Agrego un valor se "sentimiento" al campo detalle
sentiment = sentiment_analysis.SentimentAnalysisSpanish()
geo_dataP_cabaSn['valorDetalle'] = geo_dataP_cabaSn["description"].apply(lambda x: x if x is np.NaN else sentiment.sentiment(x))

In [78]:
geo_dataP_cabaSn.head(9)

,operation,property_type,place_name,lat,lon,geometry,price,currency,price_aprox_local_currency,price_aprox_usd,...,price_usd_per_m2,price_per_m2,floor,rooms,expenses,description,caracteristicas,distancia_min,distanciaSubte,valorDetalle
0,sell,PH,Mataderos,-34.661824,-58.508839,POINT (-58.50884 -34.66182),62000.0,USD,1093959.00,62000.0,...,1127.272727,1550.000000,NaN,NaN,NaN,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",|Piscina|,0.0500,Medio,0.572144
2,sell,apartment,Mataderos,-34.652262,-58.522982,POINT (-58.52298 -34.65226),72000.0,USD,1270404.00,72000.0,...,1309.090909,1309.090909,NaN,NaN,NaN,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,|Piscina|,0.0575,Lejos,0.998854
7,sell,apartment,Belgrano,-34.559873,-58.443362,POINT (-58.44336 -34.55987),138000.0,USD,2434941.00,138000.0,...,3066.666667,3450.000000,NaN,NaN,NaN,EXCELENTE MONOAMBIENTE A ESTRENAR AMPLIO SUPER...,|Piscina|Estrenar|,0.0102,Medio,0.999974
8,sell,apartment,Belgrano,-34.559873,-58.443362,POINT (-58.44336 -34.55987),195000.0,USD,3440677.50,195000.0,...,3000.000000,3250.000000,NaN,NaN,NaN,EXCELENTE DOS AMBIENTES ESTRENAR AMPLIO SUPER...,|Piscina|Estrenar|,0.0102,Medio,0.999952
13,sell,apartment,Palermo Soho,NaN,NaN,POINT (nan nan),111700.0,USD,1970890.65,111700.0,...,2234.000000,3723.333333,NaN,1.0,NaN,Torre I Mondrian. 3 ambientes con terraza y d...,|Piscina|,NaN,NaN,1.000000
14,sell,apartment,Palermo Soho,NaN,NaN,POINT (nan nan),147900.0,USD,2609621.55,147900.0,...,3521.428571,4770.967742,NaN,1.0,NaN,Torre II Dalí. Ambiente unico divisible.Vitrau...,|Piscina|,NaN,NaN,1.000000
16,sell,PH,Mataderos,-34.652356,-58.501624,POINT (-58.50162 -34.65236),239000.0,USD,4217035.50,239000.0,...,1707.142857,2438.775510,NaN,4.0,NaN,Ventas Mataderos al frente Duplex 4 amb.- Plan...,|Piscina|,0.0386,Medio,0.999881
19,sell,apartment,Palermo,-34.580504,-58.405874,POINT (-58.40587 -34.58050),350000.0,USD,6175575.00,350000.0,...,3365.384615,3645.833333,NaN,3.0,NaN,Excelente semipiso al contra frente en Bulnes ...,|Piscina|,0.0094,Cerca,1.000000
21,sell,apartment,Palermo,-34.590926,-58.411665,POINT (-58.41167 -34.59093),270500.0,USD,4772837.25,270500.0,...,2292.372881,3705.479452,NaN,4.0,NaN,"EXCELENTE ZONA, MULTIPLES MEDIOS DE TRANSPORTE...",|Piscina|,0.0027,Cerca,1.000000
